In [5]:
#Spark
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark import SparkConf

#Hadoop
from hdfs import InsecureClient

#Spark SQL functions
from pyspark.sql.functions import *
from pyspark.sql.functions import from_utc_timestamp, udf, array_distinct, col, when
from pyspark.sql.functions import regexp_replace, year, month, dayofmonth, hour, format_string
from pyspark.sql.functions import monotonically_increasing_id

# Spark Datatypes
from pyspark.sql.types import StringType, TimestampType, DateType, IntegerType
from pyspark.sql.types import DoubleType, StructType, FloatType, StructField


#Pandas
import pandas as pd
import json
import emoji
import stylecloud
from collections import Counter

In [6]:
import warnings

# Suppressing the warnings
warnings.filterwarnings('ignore')

In [7]:
spark_conf = SparkConf().setMaster("local[*]").setAppName("Tweets_Hadoop")

spark = SparkSession.builder.config(conf=spark_conf).config('spark.sql.session.timeZone', 'UTC').getOrCreate()

sc = spark.sparkContext

sc.setLogLevel('ERROR')

24/05/17 22:49:12 WARN Utils: Your hostname, muhammad-Vm resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/05/17 22:49:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/17 22:49:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/17 22:49:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [10]:
df_salvo = spark.read.parquet("hdfs://localhost:9000/CA2BD/sentiment")

In [11]:
tweets_fc_df = df_salvo.withColumn('sentiment',when(col("score") > 0, '1').otherwise('0'))

In [12]:
tweets_fc_df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- user: string (nullable = true)
 |-- text: string (nullable = true)
 |-- clean_tweet: string (nullable = true)
 |-- prediction: double (nullable = true)
 |-- textblob: float (nullable = true)
 |-- vader: float (nullable = true)
 |-- score: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- sentiment: string (nullable = false)



In [14]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql import SparkSession
# Inicializar o Tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")

# Aplicar a tokenização no DataFrame
tweets_fc_df_tokenized = tokenizer.transform(tweets_fc_df)

# Exibir o resultado
tweets_fc_df_tokenized.select("text", "tokens").show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                               |tokens                                                                                                                                                                |
+---------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|I HAVE TO SAY IT. ZAC EFRON IS MY DREAM BOY! OK 

In [16]:
tweets_fc = tweets_fc_df.withColumn("RT", when(col("text").startswith("RT"), 1).otherwise(0))

In [30]:
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

from pyspark.sql.functions import year, month, dayofmonth, hour, col, mean, count, to_date
from pyspark.sql.functions import sum as spark_sum


tweets_hour = tweets_fc.groupBy(year("date").alias("year"), 
                               month("date").alias("month"),
                               dayofmonth("date").alias("day"),
                               to_date(col("date")).alias("date"),
                               hour("date").alias("hour"),
                               "sentiment") \
                      .agg(mean(col("score")).alias("hourly_score"),
                           count(col("score")).alias("count_score"),
                           spark_sum(col("RT")).alias("sum_RT"))\
                      .orderBy("year", "month", "day", "hour")
tweets_hour.printSchema()
tweets_hour.show(2)

tweets_hour_b = tweets_fc.groupBy(year("date").alias("year"), 
                               month("date").alias("month"),
                               dayofmonth("date").alias("day"),
                               to_date(col("date")).alias("date"),
                               hour("date").alias("hour"),
                               ) \
                      .agg(mean(col("score")).alias("hourly_score"),
                           count(col("score")).alias("count_score"),
                           spark_sum(col("RT")).alias("sum_RT"))\
                      .orderBy("year", "month", "day", "hour")
tweets_hour_b.printSchema()


tweets_hour_b.show(2)

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- sentiment: string (nullable = false)
 |-- hourly_score: double (nullable = true)
 |-- count_score: long (nullable = false)
 |-- sum_RT: long (nullable = true)



+----+-----+---+----------+----+---------+--------------------+-----------+------+
|year|month|day|      date|hour|sentiment|        hourly_score|count_score|sum_RT|
+----+-----+---+----------+----+---------+--------------------+-----------+------+
|2009|    4|  7|2009-04-07|   5|        0|-0.14324630610238717|         43|     0|
|2009|    4|  7|2009-04-07|   5|        1| 0.34406590051663927|         49|     0|
+----+-----+---+----------+----+---------+--------------------+-----------+------+
only showing top 2 rows

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- hourly_score: double (nullable = true)
 |-- count_score: long (nullable = false)
 |-- sum_RT: long (nullable = true)



+----+-----+---+----------+----+-------------------+-----------+------+
|year|month|day|      date|hour|       hourly_score|count_score|sum_RT|
+----+-----+---+----------+----+-------------------+-----------+------+
|2009|    4|  7|2009-04-07|   5|0.11630041264035516|         92|     0|
|2009|    4|  7|2009-04-07|   6| 0.1477770113500622|        116|     0|
+----+-----+---+----------+----+-------------------+-----------+------+
only showing top 2 rows



In [23]:
total_tweets = tweets_fc.count()
print(f"Total of Tweets:{total_tweets:,}")

days = tweets_fc.select(to_date(col("date")).alias("data")).agg(countDistinct("data").alias("total_dias")).first()["total_dias"]
mean_tweets = total_tweets / days
print(f"Average of {mean_tweets:,.0f} tweets per day ({days} total)")

Total of Tweets:100,000


Average of 2,439 tweets per day (41 total)


In [25]:
tweets_fc_pd = tweets_fc.withColumn("date", date_format("date", "yyyy-MM-dd HH:mm:ss")).toPandas()

In [26]:
def get_top(col, top_of, top_of_rename = None, sentiment = None, top_number = None, df = tweets_fc_pd):
    
    if sentiment == None:
        df = pd.DataFrame(df[col].explode().dropna().tolist())
    else: 
        df = pd.DataFrame(df[df.sentiment == sentiment][col].explode().dropna().tolist())
        
        
    if top_number == None:
        top = df[top_of].value_counts() 
    else:
        top = df[top_of].value_counts().head(top_number)
        
    df = pd.DataFrame(top)
    df.reset_index(drop = False, inplace=True)
    df = df.rename(columns={top_of : top_of if top_of_rename == None else top_of_rename,
                               'count': 'Value'})
    
    return df 

In [27]:
tweets_hour_pd = tweets_hour.toPandas()

In [28]:
dataset_scaled_EDA = tweets_hour_pd.copy()

# min max value calculation
dataset_scaled_EDA['min_hour'] = dataset_scaled_EDA.groupby(['hour','sentiment'])[['count_score']] \
                                    .transform(lambda x: x.min())
dataset_scaled_EDA['max_hour'] = dataset_scaled_EDA.groupby(['hour','sentiment'])[['count_score']] \
                                    .transform(lambda x: x.max())

# scale
dataset_scaled_EDA['hour_scaled'] = (dataset_scaled_EDA['count_score'] - dataset_scaled_EDA['min_hour'])/(dataset_scaled_EDA['max_hour'] - dataset_scaled_EDA['min_hour'])

# add info about year, week of year and day of week
dataset_scaled_EDA['day_of_week'] = [d.strftime('%A') for d in dataset_scaled_EDA['date']]
dataset_scaled_EDA['day_of_week'] = pd.Categorical(dataset_scaled_EDA['day_of_week'], 
  categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], 
  ordered=True)

#time with hour
dataset_scaled_EDA['datetime'] = pd.to_datetime(dataset_scaled_EDA['date'].astype(str) + ' ' + 
                                                pd.to_datetime(dataset_scaled_EDA['hour'], format='%H')\
                                                .dt.time.astype(str), format='%Y-%m-%d %H:%M:%S')

dataset_scaled_EDA['sentiment'] = dataset_scaled_EDA['sentiment'].apply(lambda x: 'Positive' if x == '1' else 'Negative')
dataset_scaled_EDA.head(10) 

,year,month,day,date,hour,sentiment,hourly_score,count_score,sum_RT,min_hour,max_hour,hour_scaled,day_of_week,datetime
0,2009,4,7,2009-04-07,5,Negative,-0.143246,43,0,7,134,0.283465,Tuesday,2009-04-07 05:00:00
1,2009,4,7,2009-04-07,5,Positive,0.344066,49,0,4,181,0.254237,Tuesday,2009-04-07 05:00:00
2,2009,4,7,2009-04-07,6,Negative,-0.126357,51,0,34,132,0.173469,Tuesday,2009-04-07 06:00:00
3,2009,4,7,2009-04-07,6,Positive,0.362867,65,0,38,175,0.197080,Tuesday,2009-04-07 06:00:00
4,2009,4,7,2009-04-07,7,Positive,0.334558,71,0,29,175,0.287671,Tuesday,2009-04-07 07:00:00
5,2009,4,7,2009-04-07,7,Negative,-0.118805,50,0,41,131,0.100000,Tuesday,2009-04-07 07:00:00
6,2009,4,7,2009-04-07,8,Negative,-0.109923,65,0,38,134,0.281250,Tuesday,2009-04-07 08:00:00
7,2009,4,7,2009-04-07,8,Positive,0.326535,80,0,25,188,0.337423,Tuesday,2009-04-07 08:00:00
8,2009,4,7,2009-04-07,9,Positive,0.350315,74,0,1,176,0.417143,Tuesday,2009-04-07 09:00:00
9,2009,4,7,2009-04-07,9,Negative,-0.103772,50,0,1,121,0.408333,Tuesday,2009-04-07 09:00:00


In [31]:
tweets_hour_pd_b = tweets_hour_b.toPandas()

# min max value calculation
tweets_hour_pd_b['min_hour'] = tweets_hour_pd_b.groupby(['hour'])[['count_score']] \
                                    .transform(lambda x: x.min())
tweets_hour_pd_b['max_hour'] = tweets_hour_pd_b.groupby(['hour'])[['count_score']] \
                                    .transform(lambda x: x.max())

# scale
tweets_hour_pd_b['hour_scaled'] = (tweets_hour_pd_b['count_score'] - tweets_hour_pd_b['min_hour'])/(tweets_hour_pd_b['max_hour'] - tweets_hour_pd_b['min_hour'])

# add info about year, week of year and day of week
tweets_hour_pd_b['day_of_week'] = [d.strftime('%A') for d in tweets_hour_pd_b['date']]
tweets_hour_pd_b['day_of_week'] = pd.Categorical(tweets_hour_pd_b['day_of_week'], 
  categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], 
  ordered=True)

#time with hour
tweets_hour_pd_b['datetime'] = pd.to_datetime(tweets_hour_pd_b['date'].astype(str) + ' ' + 
                                                pd.to_datetime(tweets_hour_pd_b['hour'], format='%H')\
                                                .dt.time.astype(str), format='%Y-%m-%d %H:%M:%S')

tweets_hour_pd_b['sentiment'] = tweets_hour_pd_b['hourly_score'].apply(lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Neutral')

tweets_hour_pd_b.head(10)

,year,month,day,date,hour,hourly_score,count_score,sum_RT,min_hour,max_hour,hour_scaled,day_of_week,datetime,sentiment
0,2009,4,7,2009-04-07,5,0.116300,92,0,11,315,0.266447,Tuesday,2009-04-07 05:00:00,Positive
1,2009,4,7,2009-04-07,6,0.147777,116,0,73,295,0.193694,Tuesday,2009-04-07 06:00:00,Positive
2,2009,4,7,2009-04-07,7,0.147218,121,0,83,295,0.179245,Tuesday,2009-04-07 07:00:00,Positive
3,2009,4,7,2009-04-07,8,0.130881,145,0,76,313,0.291139,Tuesday,2009-04-07 08:00:00,Positive
4,2009,4,7,2009-04-07,9,0.167216,124,0,2,295,0.416382,Tuesday,2009-04-07 09:00:00,Positive
5,2009,4,7,2009-04-07,10,0.135434,121,0,56,333,0.234657,Tuesday,2009-04-07 10:00:00,Positive
6,2009,4,7,2009-04-07,11,0.164954,135,0,39,333,0.326531,Tuesday,2009-04-07 11:00:00,Positive
7,2009,4,7,2009-04-07,12,0.084601,104,0,95,320,0.040000,Tuesday,2009-04-07 12:00:00,Positive
8,2009,4,7,2009-04-07,13,0.162243,124,0,14,333,0.344828,Tuesday,2009-04-07 13:00:00,Positive
9,2009,4,7,2009-04-07,14,0.133470,117,0,56,320,0.231061,Tuesday,2009-04-07 14:00:00,Positive


In [32]:
df = dataset_scaled_EDA.copy()

In [42]:
import pandas as pd
import dash
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash_bootstrap_templates import load_figure_template
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
from dash.dependencies import Input,Output
from dash import callback_context, Dash, html, dcc, Input, Output, dash_table

In [43]:
load_figure_template('minty')

In [44]:
title = dbc.Row(
    [
        dbc.Col(
                dbc.CardImg(src="/assets/images/CCT_Logo.jpeg", className="img-fluid rounded-start"),
            width={"size": 3},
        ),
        dbc.Col(
            [
                html.Div(
                    [
                        html.P("Twitter Analytics for", className="card-title mx-auto"),
                        html.H1("Humans Feelings", className="text-primary mx-auto"),
                    ]
                )
            ],
            width={"size": 3, "offset": 3},
            align="center",
        ),
    ]
)

In [45]:
#CARD WITH TIMELINE -------------------------------------------------------------------------------------------------
timeline = dbc.Card(
    dbc.CardBody(
        [
            dbc.CardHeader(
                html.H2("Timeline", className="text-primary text-start card-title-large align-middle"),
            ),
            
            dcc.Graph(
                id='timeline-plot',
                figure=px.line(dataset_scaled_EDA, 
                               x='date', 
                               y='count_score',
                               labels={"date": "Date", 
                                       "count_score": "Total of Tweets"},
                              ),style={'height': '350px'}
            ),
        ],
     ),
    className="shadow my-2 text-center m-2",
    style={'margin': '0 auto'},
    color="primary", 
    outline=True ,
)


In [50]:
from datetime import datetime, timedelta

tweets_fc_pd['date'] = pd.to_datetime(tweets_fc_pd['date'])
tweets_fc_pd['sentiment'] = tweets_fc_pd['sentiment'].astype(int)

#COUNT STATICS -------------------------------------------------------------------------------------------------
#avg_tweets_day = mean_tweets

total_7 = tweets_fc_pd[(tweets_fc_pd['date'] >= pd.to_datetime(tweets_fc_pd['date'].max()) - timedelta(days=7))]['score'].count()
total_30 = tweets_fc_pd[(tweets_fc_pd['date'] >= pd.to_datetime(tweets_fc_pd['date'].max()) - timedelta(days=30))]['score'].count()
total_60 = tweets_fc_pd[(tweets_fc_pd['date'] >= pd.to_datetime(tweets_fc_pd['date'].max()) - timedelta(days=60))]['score'].count()

mean_7 = tweets_fc_pd[(tweets_fc_pd['date'] >= pd.to_datetime(tweets_fc_pd['date'].max()) - timedelta(days=7))]['score'].count() / 7
mean_30 = tweets_fc_pd[(tweets_fc_pd['date'] >= pd.to_datetime(tweets_fc_pd['date'].max()) - timedelta(days=30))]['score'].count() / 30
mean_60 = tweets_fc_pd[(tweets_fc_pd['date'] >= pd.to_datetime(tweets_fc_pd['date'].max()) - timedelta(days=60))]['score'].count() / 60

text_7 = f'{total_7:,.0f} ({mean_7:,.2f} per day)'
text_30 = f'{total_30:,.0f} ({mean_30:,.2f} per day)'
text_60 = f'{total_60:,.0f} ({mean_60:,.2f} per day)'


#CARD WITH STATICS -------------------------------------------------------------------------------------------------
stat_data = dbc.CardBody([
    dbc.Row([dbc.Row(dbc.Col(html.H5("Av. Tweets: ", className="float-start"))),
             dbc.Row(dbc.Col(html.P(['{:,.0f}'.format(mean_tweets),' per day'], className="text-primary float-end")))
    ]),
    
    html.Div([
        dbc.Row(dbc.Row(dbc.Col(html.H5("Historical Data", className="text-center text-black w-100")))),

        dbc.Row([dbc.Row(dbc.Col(html.H6("Last 7 days: ", className="float-start m-1"))),
                 dbc.Row(dbc.Col(html.P(text_7, className="float-end text-primary")))        
        ]),

        dbc.Row([dbc.Row(dbc.Col(html.H6("Last 30 days: ", className="float-start m-1"))),
                 dbc.Row(dbc.Col(html.P(text_30, className="float-end text-primary")))        
        ]),

        dbc.Row([dbc.Row(dbc.Col(html.H6("Last 60 days: ", className="float-start m-1"))),
                 dbc.Row(dbc.Col(html.P(text_60, className="float-end text-primary")))        
        ]),
    ], className="bg-light bg-gradient border rounded-top")

 ],style={'height': '350px'})


#CARD WITH STATICS -------------------------------------------------------------------------------------------------
statics = dbc.Card(
    dbc.CardBody(
        [
            dbc.CardHeader(
            html.H2("Statistic", className="text-primary text-center card-title-large align-middle"),
            ),
            stat_data
        ]
     ),
    className="shadow my-2 m-2",
    style={'margin': '0 auto'},
    color="primary", 
    outline=True ,
)

In [51]:
max_score = tweets_hour_pd_b['hourly_score'].max()
min_score = tweets_hour_pd_b['hourly_score'].min()
min_date = tweets_hour_pd_b['datetime'].min()

graf_3_1 = dbc.Card(
            dbc.CardBody(
                [
                    dbc.CardHeader(
                            html.H2("Tweet Timeline - Sentiment",
                                    className="text-primary text-start card-title-large align-middle")
                    ),
                    dcc.Graph(
                        id='tweet-timeline-plot-2',
                        figure=px.scatter(
                            tweets_hour_pd_b,
                            x="datetime",
                            y="hourly_score",
                            size="count_score",
                            color="sentiment",
                            size_max=60,
                            labels={"datetime": "Date", "count_score": "Count Scaled"}
                        )\
                        # Y axes--------------------------------------------------------------------------------------
                        .update_yaxes(visible=False,
                                       showticklabels=False,
                                       zeroline=True
                        )\
                        # Max score Line------------------------------------------------------------------------------
                        .add_hline(y=max_score, 
                              line_width=0.1, 
                              #line_dash="dash", 
                              line_color="gray"
                        )\
                        # Min Score Line------------------------------------------------------------------------------
                        .add_hline(y=min_score, 
                              line_width=0.1, 
                              #line_dash="dash", 
                              line_color="gray"
                        )\
                        # Neutral Score Line--------------------------------------------------------------------------
                        .add_hline(y=0, 
                              line_width=0.1, 
                              #line_dash="dash", 
                              line_color="gray"
                        )\
                        # Max Score Annotation------------------------------------------------------------------------
                        .add_annotation(x= min_date, 
                               y=max_score,
                               text= f"<b> Most Positive </b> <br>{max_score:,.2f}",
                               #showarrow=True,
                               #arrowhead=2,
                               ax= -40,
                               ay= 0,
                               opacity=0.7,
                               #xshift=55,
                               arrowcolor="#41b6c4"
                        )\
                        # Min Score Annotation------------------------------------------------------------------------
                        .add_annotation(x= min_date, 
                               y=min_score,
                               text= f"<b> Most Negative </b> <br>{min_score:,.2f}",
                               #showarrow=True,
                               #arrowhead=2,
                               ax= -40,
                               ay= 0,
                               opacity=0.7,
                               #xshift=55,
                               arrowcolor="#41b6c4"
                        )\
                        # Neutral Score Annotation--------------------------------------------------------------------
                        .add_annotation(x= min_date, 
                               y=0,
                               text= f"<b> Neutral</b> <br> 0.00",
                               #showarrow=True,
                               #arrowhead=2,
                               ax= -40,
                               ay= 0,
                               opacity=0.7,
                               #xshift=55,
                               arrowcolor="#41b6c4"
                        )\
                        # Adjust Legend-------------------------------------------------------------------------------
                        .update_layout(
                                legend=dict(
                                    x=1,
                                    y=1,
                                    xanchor='right',
                                    yanchor='top'
                                )
                            ),

                    ),
                ]
            ),
    className="shadow my-2 text-center m-1",
    style={'margin': '0 auto'},
    color="primary", 
    outline=True ,
        )

In [54]:
perc_total = tweets_fc_pd['sentiment'].sum() / total_tweets
perc_pos_7 = tweets_fc_pd[(tweets_fc_pd['date'] >= pd.to_datetime(tweets_fc_pd['date'].max()) - timedelta(days=7))]['sentiment'].sum() / total_7
perc_pos_30 = tweets_fc_pd[(tweets_fc_pd['date'] >= pd.to_datetime(tweets_fc_pd['date'].max()) - timedelta(days=30))]['sentiment'].sum() / total_30
perc_pos_60 = tweets_fc_pd[(tweets_fc_pd['date'] >= pd.to_datetime(tweets_fc_pd['date'].max()) - timedelta(days=60))]['sentiment'].sum() / total_60
data = {
    'period' : ['7 days', '7 days', '30 days', '30 days', '60 days', '60 days', 'Total', 'Total'],
    'sentiment' : ['Positive', 'Negative','Positive', 'Negative','Positive', 'Negative','Positive', 'Negative'],
    'value': [perc_pos_7, (1-perc_pos_7), perc_pos_30, (1-perc_pos_30), perc_pos_60, (1-perc_pos_60), perc_total, (1-perc_total)]
}
df = pd.DataFrame(data)
colors = {'Positive': 'blue', 'Negative': 'rgba(128, 128, 128, 0.1)'}


graf_stat = dbc.Card(
            dbc.CardBody(
                [
                    dbc.CardHeader(
                            html.H2("Statistics",
                                    className="text-primary text-center card-title-large align-middle")
                    ),
                    dcc.Graph(
                        id='tweet-sentiment',
                        figure = px.bar(df, x='value', y='period', color='sentiment', orientation='h',
                                     color_discrete_map=colors,
                                     labels={'value': '', 'period': '', 'sentiment': ''})\
                        
                        # Adjust Legend-------------------------------------------------------------------------------
                        .update_layout(
                            showlegend=False, 
                            xaxis_showticklabels=False,
                            yaxis_showticklabels=False, 
                            xaxis_range=[0, 1],
                            #margin=dict(l=100, r=100, t=100, b=100), 
                            uniformtext_minsize=8, 
                            bargap=0.95,  
                            xaxis=dict(showticklabels=False, showgrid=False), 
                            yaxis=dict(showticklabels=False, showgrid=False)  
                        )\
                        
                        # Adjust Legend---------------------------------TOTAL----------------------------------------------
                        .add_annotation(x= 0.2, 
                               y='Total',
                               text= f"Total",
                               #showarrow=True,
                               #arrowhead=2,
                               ax= 0,
                               ay= 0,
                               opacity=1,
                               xshift=-50,
                               yshift=17,
                               arrowcolor="#41b6c4",
                               font=dict(size=24)
                    )\
                        
                        # Adjust Legend-------------------------------------------------------------------------------
                        .add_annotation(x= 1, 
                               y='Total',
                               text= f"{perc_total:,.2%}",
                               #showarrow=True,
                               #arrowhead=6,
                               ax= 10,
                               ay= -13,
                               opacity=1,
                               #xshift=0,
                               #yshift=-17,
                               #arrowcolor="#333333",
                               font=dict(size=16)
                    )\
                        
                        # Adjust Legend---------------------------------7 DAYS----------------------------------------------
                        .add_annotation(x= 0.2, 
                               y='7 days',
                               text= f"7 Days",
                               #showarrow=True,
                               #arrowhead=2,
                               ax= 0,
                               ay= 0,
                               opacity=1,
                               xshift=-60,
                               yshift=17,
                               arrowcolor="#41b6c4",
                               font=dict(size=24)
                    )\
                        
                        # Adjust Legend-------------------------------------------------------------------------------
                        .add_annotation(x= 1, 
                               y='7 days',
                               text= f"{perc_pos_7:,.2%}",
                               #showarrow=True,
                               #arrowhead=6,
                               ax= 10,
                               ay= -13,
                               opacity=1,
                               #xshift=0,
                               #yshift=-17,
                               #arrowcolor="#333333",
                               font=dict(size=16)
                    )\
                        
                        # Adjust Legend----------------------------------30 DAYS---------------------------------------------
                        .add_annotation(x= 0.2, 
                               y='30 days',
                               text= "30 Days",
                               #showarrow=True,
                               #arrowhead=2,
                               ax= 0,
                               ay= 0,
                               opacity=1,
                               xshift=-65,
                               yshift=17,
                               arrowcolor="#41b6c4",
                               font=dict(size=24)
                    )\
                        
                        # Adjust Legend-------------------------------------------------------------------------------
                        .add_annotation(x= 1, 
                               y='30 days',
                               text= f"{perc_pos_30:,.2%}",
                               #showarrow=True,
                               #arrowhead=6,
                               ax= 10,
                               ay= -13,
                               opacity=1,
                               #xshift=0,
                               #yshift=-17,
                               #arrowcolor="#333333",
                               font=dict(size=16)
                    )\
                        
                        # Adjust Legend-----------------------------------60 DAYS--------------------------------------------
                        .add_annotation(x= 0.2, 
                               y='60 days',
                               text= "60 Days",
                               #showarrow=True,
                               #arrowhead=2,
                               ax= 0,
                               ay= 0,
                               opacity=1,
                               xshift=-65,
                               yshift=17,
                               arrowcolor="#41b6c4",
                               font=dict(size=24)
                    )\
                        
                        # Adjust Legend-------------------------------------------------------------------------------
                        .add_annotation(x= 1, 
                               y='60 days',
                               text= f"{perc_pos_60:,.2%}",
                               #showarrow=True,
                               #arrowhead=6,
                               ax= 10,
                               ay= -13,
                               opacity=1,
                               #xshift=0,
                               #yshift=-17,
                               #arrowcolor="#333333",
                               font=dict(size=16)
                    )
                      
                      
                      
                        
                        
                        
                        
                        ,style={'width': '100%', 'height': '100%'}
                )
                ]
            ),className="shadow my-2 text-center m-1",
            style={'margin': '0 auto'}
)

In [55]:
app = JupyterDash(external_stylesheets=[dbc.themes.MINTY, dbc.icons.BOOTSTRAP], )

    
app.layout = dbc.Container([
    
#TITLE---------------------------------------------------------------------------------------------------------------
    dbc.Row([
        dbc.Col(title, width = 12),
    ],
    ),
    



#ROW 2----------------------------------------------------------------------------------------------------------------
    dbc.Row([
        dbc.Col(timeline, width = 8),
        dbc.Col(statics, width = 4),
    ],style={'margin': '0 auto'}
    ),
    
#ROW 3----------------------------------------------------------------------------------------------------------------
    dbc.Row([
        dbc.Col(graf_3_1, width = 8),
        dbc.Col(graf_stat, width = 4),
    ],style={'margin': '0 auto'}
    ),
    
#ROW 4----------------------------------------------------------------------------------------------------------------

     
    

#END------------------------------------------------------------------------------------------------------------------
])



def update_tables(active_tab):
    if active_tab == 'tab-1':
        user_table_data = user_tables_pos.to_dict('records')
        urls_table_data = urls_tables_pos.to_dict('records')
        src_hashtag = 'assets/images/hashtags_pos.png'
        fig_emoji = px.bar(emojis_pos, y='Emoji', x='Frequency', orientation='h')
        fig_emoji.update_layout(yaxis_tickfont=dict(size=12), yaxis=dict(autorange="reversed"))
        
    else:
        user_table_data = user_tables_neg.to_dict('records')
        urls_table_data = urls_tables_neg.to_dict('records')
        src_hashtag = 'assets/images/hashtags_neg.png'
        fig_emoji = px.bar(emojis_neg, y='Emoji', x='Frequency', orientation='h')
        fig_emoji.update_layout(yaxis_tickfont=dict(size=12), yaxis=dict(autorange="reversed"))

    return user_table_data, urls_table_data, src_hashtag, fig_emoji
            
if __name__ == "__main__":
    app.run_server(debug=True)#mode='inline')

Dash app running on http://127.0.0.1:8050/
